In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from mediawiki import MediaWiki
from bs4 import BeautifulSoup
import string
from nltk import tokenize
from nltk.tag import StanfordNERTagger
import string
from nltk.tokenize import word_tokenize

In [2]:
st = StanfordNERTagger('/Users/venkatakrishnamohansunkara/Desktop/DM/stanford-ner-2018-02-27/classifiers/english.all.3class.distsim.crf.ser.gz',
  '/Users/venkatakrishnamohansunkara/Desktop/DM/stanford-ner-2018-02-27/stanford-ner.jar',
   encoding='utf-8') 

/anaconda3/lib/python3.6/site-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


In [3]:
wikipedia = MediaWiki()

In [13]:
#df = pd.DataFrame(columns=['vil_name','sub_dist_name','dist_name','state'])
df = pd.read_csv('all names india/all_names_census_india.csv')

In [14]:
#df.loc[0] = ['etah','etah','etah','uttar pradesh']
#df.loc[1] = ['pedaparupudi','pedaparupudi','krishna','andhra pradesh']

In [19]:
df.head()

,Unnamed,State_Id,state,dist_id,dist_name,sub_dist_id,sub_dist_name,vil_id,vil_name
0,0,11,SIKKIM,0,SIKKIM,0.0,SIKKIM,0,SIKKIM
1,1,11,SIKKIM,241,North District,0.0,North District,0,North District
2,2,11,SIKKIM,241,North District,1547.0,Chungthang,0,Chungthang
3,3,11,SIKKIM,241,North District,1547.0,Chungthang,260925,Lachen
4,4,11,SIKKIM,241,North District,1547.0,Chungthang,260926,Lachung


In [18]:
df.columns = ['Unnamed','State_Id','state','dist_id','dist_name','sub_dist_id','sub_dist_name','vil_id','vil_name']

In [7]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    string = re.sub(r"\n", "", string)
    string = re.sub(r"=", "", string)
    string = re.sub(r"-", "", string)
    string = re.sub(r"/", "", string)
    return string.strip()

In [20]:
def add2file(sent,search_str,fh):
    text = nltk.word_tokenize(sent)
    ner = st.tag(text)
    words = nltk.word_tokenize(search_str.lower())
    for w in ner:
        if w[0].lower() in words:
            fh.write('%s\t%s\n'%(w[0],'LOCATION'))
        else:
            fh.write('%s\t%s\n'%(w[0],w[1]))
    fh.write('\n')

In [21]:
def searchWordinSentence(word,sentence):
    pattern = re.compile(' '+word+' |^'+word+' | '+word+' $')
    if re.search(pattern,sentence.lower()):
        return True

In [22]:
table = str.maketrans({key: None for key in string.punctuation})

In [23]:
df2 = df.sample(1000)

In [24]:
fh = open('sub_test.txt','w+')
for i,row in df2.iterrows():
    search_str = row['vil_name']+" "+"village"+" "+row['sub_dist_name']+" "+row['dist_name']+" "+row['state']+" "+"state"+" "+"India"
    split_str = row['vil_name'] + " "+ row['sub_dist_name']+" "+row['dist_name']+" "+row['state']+" "+"India"
    main_search_word = row['vil_name'].lower()
    search_str = search_str.strip()
    split_str = split_str.strip().lower()
    searches = wikipedia.search(search_str)
    no_sents = 0
    print(len(searches))
    if len(searches) > 0:
        i = 0
        no_searches = len(searches)
        while no_sents < 3 and i<no_searches:
            print(no_sents)
            page = wikipedia.page(searches[i])
            text = BeautifulSoup(page.content)
            text = clean_str(text.get_text())
            sentences = tokenize.sent_tokenize(text)
            for j in sentences:
                if searchWordinSentence(main_search_word,j) and no_sents<3:
                    no_sents+=1;
                    add2file(j,split_str,fh)
                else:
                    continue
            i+=1
    if no_sents<3:
        sent1 = str(row['vil_name']) + " is a village present in the sub-district named " +str(row['sub_dist_name'])+" which is present in the " +str(row['dist_name'])+" district of "+ str(row['state']) +" in the country of India."
        sent2 = "India contains a village named "+str(row['vil_name'])+" in the sub-district of "+str(row['sub_dist_name'])+" which is present in the district named "+str(row['dist_name'])+" in the great state of "+str(row['state'])
        add2file(sent1.strip(),split_str,fh)
        add2file(sent2.strip(),split_str,fh)
    else:
        print('Found more that 3 :)')

0
0
0
0
10
0


/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Found more that 3 :)
0
0
0
0
10
0


/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


1


...

0
0
0
0
1
0


/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


0
0
2
0
0


/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


0
0
0
0
0
0
0


In [25]:
fh.close()

In [85]:
df_test = pd.DataFrame(data=[["angul","","","angul","odisha"],
                            ["elamarru","elamarru","pedaparapudi","krishna","Andhra Pradesh"]],columns=['main_search_word','vil_name','sub_dist_name','dist_name','state'])

In [147]:
pwd

'/Users/venkatakrishnamohansunkara/Desktop/DM'

In [3]:
import glob
import os

In [130]:
df_all = pd.DataFrame()

In [131]:
for f in glob.glob('Village_names, census/*.xls'):
    data = pd.read_excel(f)
    df_all = df_all.append(data)

In [132]:
df_up = pd.read_csv('Village_names, census/ods/Rdir_2011_09_UTTAR_PRADESH.csv')

In [133]:
df_mp = pd.read_csv('Village_names, census/Rdir_2011_23_MADHYA_PRADESH.csv')

In [135]:
df_all = df_all.append(df_mp)

In [136]:
df_all = df_all.append(df_up)

In [138]:
df_all.shape

(652546, 8)

In [139]:
df_all.to_csv('all_names_census_india.csv')

In [252]:
df_all.shape

(652546, 8)

In [143]:
df_all = df_all.rename(index=str,columns={"STATE NAME":"state","DISTRICT NAME":"dist_name","SUB-DISTRICT NAME":"sub_dist_name","Area Name":"vil_name"})

In [3]:
test_st = StanfordNERTagger('/Users/venkatakrishnamohansunkara/Desktop/DM/stanford_NER/ner-model-india.gaz.ser.gz',
  '/Users/venkatakrishnamohansunkara/Desktop/DM/stanford_NER/stanford-ner.jar',
   encoding='utf-8') 

/anaconda3/lib/python3.6/site-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


In [9]:
test_txt = "a protest occured about 50 miles south of Delhi at a place named pedaparupudi in etah district of Uttar Pradesh"
#test_txt = "A man was arrested for hurling a shoe at Delhi CM and AAP national convener Arvind Kejriwal when he was addressing a public meeting in Rohtak on Sunday"

In [10]:
test_tokens = nltk.word_tokenize(test_txt)
# Using Default stanford tagger
st_tags = st.tag(test_tokens)
# Using Trained stanford tagger
ans = test_st.tag(test_tokens)

In [11]:
st_tags

[('a', 'O'),
 ('protest', 'O'),
 ('occured', 'O'),
 ('about', 'O'),
 ('50', 'O'),
 ('miles', 'O'),
 ('south', 'O'),
 ('of', 'O'),
 ('Delhi', 'LOCATION'),
 ('at', 'O'),
 ('a', 'O'),
 ('place', 'O'),
 ('named', 'O'),
 ('pedaparupudi', 'O'),
 ('in', 'O'),
 ('etah', 'O'),
 ('district', 'O'),
 ('of', 'O'),
 ('Uttar', 'LOCATION'),
 ('Pradesh', 'LOCATION')]

In [12]:
ans

[('a', 'O'),
 ('protest', 'O'),
 ('occured', 'O'),
 ('about', 'O'),
 ('50', 'O'),
 ('miles', 'O'),
 ('south', 'O'),
 ('of', 'O'),
 ('Delhi', 'LOCATION'),
 ('at', 'O'),
 ('a', 'O'),
 ('place', 'O'),
 ('named', 'O'),
 ('pedaparupudi', 'LOCATION'),
 ('in', 'O'),
 ('etah', 'LOCATION'),
 ('district', 'O'),
 ('of', 'O'),
 ('Uttar', 'LOCATION'),
 ('Pradesh', 'LOCATION')]

In [187]:
f2 = open('india_census_gazette.txt','w+')
count = 0
for i,row in df_all.iterrows():
    f2.write('%s\t%s\n'%('Location',row['vil_name']))
    count+=1
f2.close()

In [199]:
nga_all = pd.read_csv('all names india/in/in.txt',sep='\t')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (16,18,32,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [208]:
f2 = open('india_nga_gazette.txt','w+')
count = 0
for i,row in nga_all.iterrows():
    f2.write('%s\t%s\n'%('Location',row['FULL_NAME_ND_RO']))
    count+=1
f2.close()

In [253]:
sum(1 for line in open('stanford_NER/india_nga_gazette.txt'))

662519

In [230]:
fn = open('stanford_NER/all_india_sub.txt','w+')
count = 0
for line in open('stanford_NER/all_india.txt','r'):
    #print(count)
    if count <1000000:
        fn.write(line)
        count+=1
    if count == 1000000:
        break
fn.close()

In [35]:
news_data = pd.read_csv('/Users/venkatakrishnamohansunkara/Desktop/DM/GDELT_2017/large_dataset/unrest_2017_ie_toi.csv', sep=',')

In [56]:
test_data = news_data.iloc[100].content + "A new event took place in the angul division of adoni"

In [58]:
test_sent = nltk.sent_tokenize(test_data)

In [64]:
stanford = []
new_model = []
i=1;
for sent in test_sent:
    print(i);
    i+=1;
    test_tags = nltk.word_tokenize(sent)
    original_tags = st.tag(test_tags)
    new_model_tags = test_st.tag(test_tags)
    for e,t in original_tags:
        if t == 'LOCATION':
            stanford = stanford + [e];
    for e,t in new_model_tags:
        if t == 'LOCATION':
           new_model = new_model + [e];

1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [65]:
stanford

['CHANDIGARH',
 'Delhi',
 'Rohtak',
 'Bhiwani',
 'Haryana',
 'Haryana',
 'Rohtak',
 'Haryana']

In [66]:
new_model

['Delhi',
 'Rohtak',
 'Sunday',
 'Dadri',
 'Bhiwani',
 'Rohtak',
 'Kejriwal',
 'Prime',
 'Minister',
 'Modi',
 'Swiss']

In [48]:
test_tags = nltk.word_tokenize(test_sent)

In [49]:
original_tags = st.tag(test_tags)

In [50]:
new_model_tags = test_st.tag(test_tags)

In [51]:
for e,t in original_tags:
    if t == 'LOCATION':
        print(e)

In [52]:
for e,t in new_model_tags:
    if t == 'LOCATION':
        print(e)

In [57]:
test_data

'CHANDIGARH: A man was arrested for hurling a shoe at Delhi CM and AAP national convener Arvind Kejriwal when he was addressing a public meeting in Rohtak on Sunday. The shoe, however, missed its target.The incident happened when Kejriwal was speaking at ‘Tijori Tod Bhanda Fod\' rally against demonetisation, during which he targeted Prime Minister Narendra Modi. The shoe-thrower has been identified as Vikas Kumar from Dadri in Bhiwani district. He is a graduate and unemployed, a police official said. AAP workers and cops in civilian clothes overpowered Vikas and handed him over to police at Urban Estate station. SHO Ramesh Kumar said the shoe fell some distance from the main stage. "He has been taken into custody. During interrogation, he said he was hurt by Kejriwal\'s stand on the SYL issue which went against Haryana\'s interests. Though we have not received any formal complaint from organizers so far, he has been put under preventive arrest for breach of peace," the SHO said.Asked w